# This Script will be used to transform from PDM_Lite format to KITTI dataset format

In [515]:
# testing parsing xml
import xml.etree.ElementTree as ET
import sys

XML_PATH = "/home/nupdm/workspace/pdm_lite/leaderboard/data/routes_devtest.xml"

data = ET.parse(XML_PATH)
root = data.getroot()

for route in root.findall("route"):
    route_id = route.get("id")
    town = route.get("town")
    print(f"Route ID: {route_id}, Town: {town}")

    for scenario in route.find("scenarios").findall("scenario"):
        name = scenario.get("name")
        scenario_type = scenario.get("type")
        print(f"Scenario: {name} of route {route_id} in town {town}")
        trigger_point = scenario.find("trigger_point")
        x, y, z = trigger_point.get("x"), trigger_point.get("y"), trigger_point.get("z")
        yaw = trigger_point.get("yaw")
        print(f"  - Scenario: {name} ({scenario_type}) at ({x}, {y}, {z}) Yaw: {yaw}")


Route ID: 0, Town: Town12
Scenario: ParkingExit_1 of route 0 in town Town12
  - Scenario: ParkingExit_1 (ParkingExit) at (983.5, 5382.2, 371) Yaw: 90
Scenario: SignalizedJunctionLeftTurn_1 of route 0 in town Town12
  - Scenario: SignalizedJunctionLeftTurn_1 (SignalizedJunctionLeftTurn) at (571.1, 5579.4, 370) Yaw: 180
Scenario: NonSignalizedJunctionRightTurn_1 of route 0 in town Town12
  - Scenario: NonSignalizedJunctionRightTurn_1 (NonSignalizedJunctionRightTurn) at (546.9, 5982.8, 362) Yaw: 77
Scenario: Accident_1 of route 0 in town Town12
  - Scenario: Accident_1 (Accident) at (427.2, 6086.8, 359) Yaw: 144
Scenario: NonSignalizedJunctionLeftTurn_1 of route 0 in town Town12
  - Scenario: NonSignalizedJunctionLeftTurn_1 (NonSignalizedJunctionLeftTurn) at (294.0, 6153.2, 358) Yaw: 249
Scenario: ControlLoss_1 of route 0 in town Town12
  - Scenario: ControlLoss_1 (ControlLoss) at (119.7, 6100.9, 358) Yaw: 179
Scenario: ConstructionObstacleTwoWays_1 of route 0 in town Town12
  - Scenario:

### Current Format
boxes
measurements
CAM_FRONT
CAM_FRONT_LEFT
CAM_FRONT_RIGHT
CAM_BACK
CAM_BACK_LEFT
CAM_BACK_RIGHT
lidar
lidar_semantic
bev_semantics

### KITTI Format
kitti
    |- training
        |- calib (#7481 .txt)
        |- image_2 (#7481 .png)
        |- label_2 (#7481 .txt)
        |- velodyne (#7481 .bin)
    |- testing
        |- calib (#7518 .txt)
        |- image_2 (#7518 .png)
        |- velodyne (#7518 .bin)

### New Format
kitti
    |- training
        |- calib
        |- CAM_FRONT
        |- CAM_FRONT_LEFT
        |- CAM_FRONT_RIGHT
        |- CAM_BACK
        |- CAM_BACK_LEFT
        |- CAM_BACK_RIGHT
        |- labels
        |- points (.bin)
    |- testing
        |- calib 
        |- CAM_FRONT
        |- CAM_FRONT_LEFT
        |- CAM_FRONT_RIGHT
        |- CAM_BACK
        |- CAM_BACK_LEFT
        |- CAM_BACK_RIGHT
        |- points (.bin)

In [516]:
import os
import gzip
import json
import numpy as np
import laspy
import matplotlib.pyplot as plt

# ITEM = "0040"

# MASTER_DATAROOT = "/home/nupdm/Datasets/nuPDM/routes_training_test"
MASTER_DATAROOT = "/home/nupdm/Datasets/nuPDM/routes_training_mini"
# MASTER_DATAROOT = "/home/nupdm/Datasets/nuPDM/routes_training"
single_route = True

# list all folders in order
folder_names = []
for folder in sorted(os.listdir(MASTER_DATAROOT)):
    if "route" not in folder:
        continue
    route_number = folder.split("_")[1]
    folder_name = os.path.join(MASTER_DATAROOT, folder)
    folder_names.append(folder_name)

print(folder_names)

['/home/nupdm/Datasets/nuPDM/routes_training_mini/route_0_routes_training_date_03_04_13_55_31']


In [517]:
# np.set_printoptions(precision=4, suppress=True)

# print("Lidar to Ego:")
# print(lidar2ego)
# print("\n")

# print("Ego to cam_front:")
# print(ego2cam_front)
# print("\n")

# print("Ego to cam_back:")
# print(ego2cam_back)
# print("\n")

# print("Ego to cam_front_left:")
# print(ego2cam_front_left)
# print("\n")

# print("Ego to cam_front_right:")
# print(ego2cam_front_right)
# print("\n")

# print("Ego to cam_back_left:")
# print(ego2cam_back_left)
# print("\n")

# print("Ego to cam_back_right:")
# print(ego2cam_back_right)
# print("\n")

In [518]:
print("Lidar to ego:")
print(lidar2ego)
print("\n")

print("Ego to cam_front:")
print(ego2cam_front)
print("\n")

print("Lidar to cam_front:")
print(lidar2cam_front)
print("\n")

print("Lidar to cam_back:")
print(lidar2cam_back)
print("\n")

print("Ego to cam_back:")
print(ego2cam_back)
print("\n")

Lidar to ego:
[[ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


Ego to cam_front:
[[ 0.     1.     0.    -0.   ]
 [ 0.     0.    -1.     1.51 ]
 [ 1.     0.     0.    -0.756]
 [ 0.     0.     0.     1.   ]]


Lidar to cam_front:
[[ 0.    -1.     0.     0.   ]
 [ 0.     0.    -1.     1.51 ]
 [ 1.     0.     0.    -0.756]
 [ 0.     0.     0.     1.   ]]


Lidar to cam_back:
[[ 0.    1.    0.    0.  ]
 [ 0.    0.   -1.    1.68]
 [-1.    0.    0.   -1.1 ]
 [ 0.    0.    0.    1.  ]]


Ego to cam_back:
[[ 0.   -1.    0.   -0.  ]
 [ 0.    0.   -1.    1.68]
 [-1.   -0.    0.   -1.1 ]
 [ 0.    0.    0.    1.  ]]




In [519]:
P_CAM_FRONT

array([[1142.5184,    0.    ,  800.    ],
       [   0.    , 1142.5184,  450.    ],
       [   0.    ,    0.    ,    1.    ]])

In [520]:
# Now generate calib, labels and points for each folder

def create_transformation_matrix(rotation_matrix, translation_vector):
    """
    Crea una matriz de transformación homogénea 4x4 a partir de
    una matriz de rotación 3x3 y un vector de traslación 3x1
    """
    transform = np.eye(4)
    transform[:3, :3] = rotation_matrix
    transform[:3, 3] = translation_vector
    return transform

for folder in folder_names:
    DATAROOT = folder
    BOXES_PATH = os.path.join(DATAROOT, "boxes")
    MEASUREMENTS_PATH = os.path.join(DATAROOT, "measurements")
    print("DATAROOT: ", DATAROOT)

    # read number of files in the folders
    n_files = len([name for name in os.listdir(BOXES_PATH) if os.path.isfile(os.path.join(BOXES_PATH, name))])
    print("Number of files in boxes folder: ", n_files)

    for ITEM in range(n_files):
        ITEM = str(ITEM).zfill(4)
        # load measurements
        with gzip.open(os.path.join(MEASUREMENTS_PATH, ITEM + ".json.gz"), "rb") as f:
            measurements = json.load(f)

        # load boxes
        with gzip.open(os.path.join(BOXES_PATH, ITEM + ".json.gz"), "rb") as f:
            boxes = json.load(f)

        P_CAM_FRONT = np.array(boxes[0]["cam_front_K"])
        P_CAM_FRONT_LEFT = np.array(boxes[0]["cam_front_left_K"])
        P_CAM_FRONT_RIGHT = np.array(boxes[0]["cam_front_right_K"])
        P_CAM_BACK = np.array(boxes[0]["cam_back_K"])
        P_CAM_BACK_LEFT = np.array(boxes[0]["cam_back_left_K"])
        P_CAM_BACK_RIGHT = np.array(boxes[0]["cam_back_right_K"])

        # ------------------------------- Transformation matrixes -------------------------------------
        # LIDAR
        # lidar_translation = np.array([0, 0, -1.85])
        lidar_translation = np.array([0, 0, 0])
        lidar_rotation = np.array([
            [1,  0, 0],  # x_ego = x_lidar
            [0, -1, 0],  # y_ego = -y_lidar (right = -left)
            [0,  0, 1]   # z_ego = z_lidar
        ])
        lidar2ego = create_transformation_matrix(lidar_rotation, lidar_translation)
        ego2lidar = np.linalg.inv(lidar2ego)

        # CAMERAS
        ego2cams_align = np.array([
            [0, 1, 0],
            [0, 0, -1],
            [1, 0, 0]
        ])
        # CAM_FRONT
        ego2cam_front_traslation = np.array([-0.0, 1.51, -0.756])
        # ego2cam_front_traslation = np.array([-0.0, 1.51, -0.756]) # NO TIENE SENTIDO
        ego2cam_front_yaw = 0.0
        ego2cam_front_rot = np.array([
            [np.cos(ego2cam_front_yaw), -np.sin(ego2cam_front_yaw), 0],
            [np.sin(ego2cam_front_yaw),  np.cos(ego2cam_front_yaw), 0],
            [0, 0, 1]
        ])

        ego2cam_front_rotation = np.dot(ego2cams_align, ego2cam_front_rot)
        ego2cam_front = create_transformation_matrix(ego2cam_front_rotation, ego2cam_front_traslation)
        cam_front2ego = np.linalg.inv(ego2cam_front)

        # CAM_BACK
        # ego2cam_back_traslation = np.array([-0.0, 1.68, 1.1])
        ego2cam_back_traslation = np.array([-0.0, 1.68, -1.1])
        ego2cam_back_yaw = np.deg2rad(180.0)
        ego2cam_back_rot = np.array([
            [np.cos(ego2cam_back_yaw), -np.sin(ego2cam_back_yaw), 0],
            [np.sin(ego2cam_back_yaw),  np.cos(ego2cam_back_yaw), 0],
            [0, 0, 1]
        ])

        ego2cam_back_rot_final = np.dot(ego2cams_align, ego2cam_back_rot)
        ego2cam_back = create_transformation_matrix(ego2cam_back_rot_final, ego2cam_back_traslation)
        cam_back2ego = np.linalg.inv(ego2cam_back)

        # CAM_FRONT_LEFT
        # ego2cam_front_left_traslation = np.array([0.5, 1.51, -0.6])
        ego2cam_front_left_traslation = np.array([0.6, 1.51, 0.5])
        ego2cam_front_left_yaw = np.deg2rad(55.0)
        ego2cam_front_left_rot = np.array([
            [np.cos(ego2cam_front_left_yaw), -np.sin(ego2cam_front_left_yaw), 0],
            [np.sin(ego2cam_front_left_yaw),  np.cos(ego2cam_front_left_yaw), 0],
            [0, 0, 1]
        ])
        ego2cam_front_left_rotation = np.dot(ego2cams_align, ego2cam_front_left_rot)
        ego2cam_front_left = create_transformation_matrix(ego2cam_front_left_rotation, ego2cam_front_left_traslation)
        cam_front_left2ego = np.linalg.inv(ego2cam_front_left)

        # CAM_FRONT_RIGHT
        ego2cam_front_right_traslation = np.array([0.6, 1.51, -0.5])
        ego2cam_front_right_yaw = np.deg2rad(-55.0)
        ego2cam_front_right_rot = np.array([
            [np.cos(ego2cam_front_right_yaw), -np.sin(ego2cam_front_right_yaw), 0],
            [np.sin(ego2cam_front_right_yaw),  np.cos(ego2cam_front_right_yaw), 0],
            [0, 0, 1]
        ])
        ego2cam_front_right_rotation = np.dot(ego2cams_align, ego2cam_front_right_rot)
        ego2cam_front_right = create_transformation_matrix(ego2cam_front_right_rotation, ego2cam_front_right_traslation)
        cam_front_right2ego = np.linalg.inv(ego2cam_front_right)

        # CAM_BACK_LEFT
        # egocam_back_left_traslation = np.array([0.5, 1.58, -0.07])
        egocam_back_left_traslation = np.array([0.5, 1.58, 0.07])
        ego2cam_back_left_yaw = np.deg2rad(110.0)
        ego2cam_back_left_rot = np.array([
            [np.cos(ego2cam_back_left_yaw), -np.sin(ego2cam_back_left_yaw), 0],
            [np.sin(ego2cam_back_left_yaw),  np.cos(ego2cam_back_left_yaw), 0],
            [0, 0, 1]
        ])
        ego2cam_back_left_rotation = np.dot(ego2cams_align, ego2cam_back_left_rot)
        ego2cam_back_left = create_transformation_matrix(ego2cam_back_left_rotation, egocam_back_left_traslation)
        cam_back_left2ego = np.linalg.inv(ego2cam_back_left)            

        # CAM_BACK_RIGHT
        # ego2cam_back_right_traslation = np.array([-0.5, 1.58, -0.07])
        ego2cam_back_right_traslation = np.array([-0.5, 1.58, 0.07])
        ego2cam_back_right_yaw = np.deg2rad(-110.0)
        ego2cam_back_right_rot = np.array([
            [np.cos(ego2cam_back_right_yaw), -np.sin(ego2cam_back_right_yaw), 0],
            [np.sin(ego2cam_back_right_yaw),  np.cos(ego2cam_back_right_yaw), 0],
            [0, 0, 1]
        ])
        ego2cam_back_right_rotation = np.dot(ego2cams_align, ego2cam_back_right_rot)
        ego2cam_back_right = create_transformation_matrix(ego2cam_back_right_rotation, ego2cam_back_right_traslation)
        cam_back_right2ego = np.linalg.inv(ego2cam_back_right)
        
        lidar2cam_front = np.dot(ego2cam_front, lidar2ego)
        lidar2cam_front_left = np.dot(ego2cam_front_left, lidar2ego)
        lidar2cam_front_right = np.dot(ego2cam_front_right, lidar2ego)
        lidar2cam_back = np.dot(ego2cam_back, lidar2ego)
        lidar2cam_back_left = np.dot(ego2cam_back_left, lidar2ego)
        lidar2cam_back_right = np.dot(ego2cam_back_right, lidar2ego)

        # create the directory if it does not exist
        os.makedirs(os.path.join(DATAROOT, "calib"), exist_ok=True)

        with open(os.path.join(DATAROOT, "calib", ITEM + ".txt"), "w") as f:
            f.write("CAM_FRONT_K: " + " ".join(map(str, P_CAM_FRONT.flatten())) + "\n")
            f.write("CAM_FRONT_LEFT_K: " + " ".join(map(str, P_CAM_FRONT_LEFT.flatten())) + "\n")
            f.write("CAM_FRONT_RIGHT_K: " + " ".join(map(str, P_CAM_FRONT_RIGHT.flatten())) + "\n")
            f.write("CAM_BACK_K: " + " ".join(map(str, P_CAM_BACK.flatten())) + "\n")
            f.write("CAM_BACK_LEFT_K: " + " ".join(map(str, P_CAM_BACK_LEFT.flatten())) + "\n")
            f.write("CAM_BACK_RIGHT_K: " + " ".join(map(str, P_CAM_BACK_RIGHT.flatten())) + "\n")
            f.write("LIDAR2CAM_FRONT: " + " ".join(map(str, lidar2cam_front.flatten())) + "\n")
            f.write("LIDAR2CAM_FRONT_LEFT: " + " ".join(map(str, lidar2cam_front_left.flatten())) + "\n")
            f.write("LIDAR2CAM_FRONT_RIGHT: " + " ".join(map(str, lidar2cam_front_right.flatten())) + "\n")
            f.write("LIDAR2CAM_BACK: " + " ".join(map(str, lidar2cam_back.flatten())) + "\n")
            f.write("LIDAR2CAM_BACK_LEFT: " + " ".join(map(str, lidar2cam_back_left.flatten())) + "\n")
            f.write("LIDAR2CAM_BACK_RIGHT: " + " ".join(map(str, lidar2cam_back_right.flatten())) + "\n")

        # print("Calib file created")

        # now go for the labels with format [<object_type> <truncation> <occlusion> <alpha> <left> <top> <right> <bottom> <height> <width> <length> <x> <y> <z> <rotation_y> <num_points>]
        # x, y, z are the coordinates of the center of the object
        # dx, dy, dz are the dimensions of the object
        # yaw is the rotation of the object
        # category is the category of the object

        # las dimensiones de nuPDM vienen en l/2, w/2, h/2
        # las dimensiones de KITTI vienen en h, w, l

        items = []

        # VERY IMPORTANT
        # PDM location format is:
            # x: forward
            # y: right
            # z: up
        # KITTI location format is in camera coordinates:
            # x: right
            # y: down
            # z: forward
            
        # KITTI LiDAR format is:
            # x: forward
            # y: left
            # z: up
        # KITTI IMU format is:
            # x: forward
            # y: left
            # z: up            

        # The center of the 3D bbox will be in x, y, z, since we will have our objects in origint=(0.5, 0.5, 0)

        for i, box in enumerate(boxes):
            if box["class"] == "car" or box["class"] == "static_car": # coche
                # CARLA base types are car, truck, van, bus, motorcycle, bicycle TODO: Change this to add more types
                if ("base_type" in box):
                    base_type = box["base_type"]
                else:
                    base_type = "car"

                # items.append([base_type, 
                #             np.array(box["extent"][1])*2,   # Antes esto era 1, 0, 2, IMPORTANTE
                #             np.array(box["extent"][0])*2, 
                #             np.array(box["extent"][2])*2, 
                #             box["position"][0],             # Antes esto era 0, 1, 2, IMPORTANTE
                #             -box["position"][1], 
                #             box["position"][2], ######## CHECK THIS
                #             float(box["yaw"]), # - np.pi/2, 
                #             box["num_points"]])
                

                # yaw = -float(box["yaw"]) - np.pi/2

                #normalize between -pi, pi
                # if yaw > np.pi:
                #     yaw -= 2*np.pi
                # if yaw < -np.pi:
                #     yaw += 2*np.pi

                items.append([base_type, 
                            np.array(box["extent"][1])*2,   # h
                            np.array(box["extent"][2])*2,   # w # creo que estos 2 deberían ser al revés (mirar PDMLite)
                            np.array(box["extent"][0])*2,   # l
                            box["position"][0],             # x
                            -box["position"][1],            # -y
                            box["position"][2],             # z
                            -float(box["yaw"]),              # yaw
                            box["num_points"]])

            elif box["class"] == "walker":
                    items.append(["walker", 
                                np.array(box["extent"][1])*2,   # Antes esto era 1, 0, 2, IMPORTANTE
                                np.array(box["extent"][2])*2, 
                                np.array(box["extent"][0])*2, 
                                box["position"][0],             # Antes esto era 0, 1, 2, IMPORTANTE
                                -box["position"][1], 
                                box["position"][2], ########CHECK THIS
                                -float(box["yaw"]), 
                                box["num_points"]])
            elif box["class"] == "traffic_light_vqa":
                    items.append(["traffic_light", 
                                np.array(box["extent"][1])*2,   # Antes esto era 1, 0, 2, IMPORTANTE
                                np.array(box["extent"][2])*2, 
                                np.array(box["extent"][0])*2, 
                                box["position"][0],             # Antes esto era 0, 1, 2, IMPORTANTE
                                -box["position"][1], 
                                box["position"][2], ########CHECK THIS
                                -float(box["yaw"]), 
                                box["num_points"]])
            elif box["class"] == "stop_sign_vqa":
                    items.append(["stop_sign", 
                                np.array(box["extent"][1])*2,   # Antes esto era 1, 0, 2, IMPORTANTE
                                np.array(box["extent"][2])*2, 
                                np.array(box["extent"][0])*2, 
                                box["position"][0],             # Antes esto era 0, 1, 2, IMPORTANTE
                                -box["position"][1], 
                                box["position"][2], ########CHECK THIS
                                -float(box["yaw"]), 
                                box["num_points"]])
            elif box["class"] == "static_trafficwarning":
                    items.append(["static_trafficwarning", 
                                np.array(box["extent"][1])*2,   # Antes esto era 1, 0, 2, IMPORTANTE
                                np.array(box["extent"][2])*2, 
                                np.array(box["extent"][0])*2, 
                                box["position"][0],             # Antes esto era 0, 1, 2, IMPORTANTE
                                -box["position"][1], 
                                box["position"][2], ########CHECK THIS
                                -float(box["yaw"]), 
                                box["num_points"]])
            else:
                # print("Unknown class: ", box["class"])
                pass
        
        # create the directory if it does not exist
        os.makedirs(os.path.join(DATAROOT, "labels"), exist_ok=True)

        with open(os.path.join(DATAROOT, "labels", ITEM + ".txt"), "w") as f:
            for item in items:
                f.write(" ".join(map(str, item)) + "\n")

        # print("Labels created")
        # open pointcloud
        PATH_LIDAR = os.path.join(DATAROOT, "lidar", ITEM + ".laz")

        with laspy.open(PATH_LIDAR) as f:
            points = f.read()

        x = points['X']/1000 + f.header.offset[0]
        y = -(points['Y']/1000 + f.header.offset[1])
        z = points['Z']/1000 + f.header.offset[2]
        i = points['intensity']

        # save in points folder as .bin
        os.makedirs(os.path.join(DATAROOT, "points"), exist_ok=True)

        with open(os.path.join(DATAROOT, "points", ITEM + ".bin"), "wb") as f:
            np.array([x, y, z, i]).T.astype(np.float32).tofile(f)

        # print("Points saved")

        # open bin file
        # with open(os.path.join(DATAROOT, "points", ITEM + ".bin"), "rb") as f:
        #     points = np.fromfile(f, dtype=np.float32).reshape(-1, 4)

        # # show in bev using plt
        # fig = plt.figure(figsize=(10, 10))
        # ax = fig.add_subplot(111)
        # ax.scatter(points[:, 0], points[:, 1], c=points[:, 2], cmap='gray', s=1)
        # plt.xlim([-30, 30])
        # plt.ylim([-30, 30])
        # plt.show()



DATAROOT:  /home/nupdm/Datasets/nuPDM/routes_training_mini/route_0_routes_training_date_03_04_13_55_31
Number of files in boxes folder:  100


In [521]:
np.set_printoptions(precision=4, suppress=True)

print("Lidar to Ego:")
print(lidar2ego)
print("\n")

print("Ego to cam_front:")
print(ego2cam_front)
print("\n")

print("Ego to cam_back:")
print(ego2cam_back)
print("\n")

print("Ego to cam_front_left:")
print(ego2cam_front_left)
print("\n")

print("Ego to cam_front_right:")
print(ego2cam_front_right)
print("\n")

print("Ego to cam_back_left:")
print(ego2cam_back_left)
print("\n")

print("Ego to cam_back_right:")
print(ego2cam_back_right)
print("\n")

# print lidar2cam_front
print("Lidar to cam_front:")
print(lidar2cam_front)

Lidar to Ego:
[[ 1.  0.  0.  0.]
 [ 0. -1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]


Ego to cam_front:
[[ 0.     1.     0.    -0.   ]
 [ 0.     0.    -1.     1.51 ]
 [ 1.     0.     0.    -0.756]
 [ 0.     0.     0.     1.   ]]


Ego to cam_back:
[[ 0.   -1.    0.   -0.  ]
 [ 0.    0.   -1.    1.68]
 [-1.   -0.    0.   -1.1 ]
 [ 0.    0.    0.    1.  ]]


Ego to cam_front_left:
[[ 0.8192  0.5736  0.      0.6   ]
 [ 0.      0.     -1.      1.51  ]
 [ 0.5736 -0.8192  0.     -0.5   ]
 [ 0.      0.      0.      1.    ]]


Ego to cam_front_right:
[[-0.8192  0.5736  0.      0.6   ]
 [ 0.      0.     -1.      1.51  ]
 [ 0.5736  0.8192  0.      0.5   ]
 [ 0.      0.      0.      1.    ]]


Ego to cam_back_left:
[[ 0.9397 -0.342   0.      0.5   ]
 [ 0.      0.     -1.      1.58  ]
 [-0.342  -0.9397  0.      0.07  ]
 [ 0.      0.      0.      1.    ]]


Ego to cam_back_right:
[[-0.9397 -0.342   0.     -0.5   ]
 [ 0.      0.     -1.      1.58  ]
 [-0.342   0.9397  0.      0.07  ]
 [ 0.   

In [522]:
# for each folder, get 75% of files randomly and move them to train, the rest to val
import random

for folder in folder_names:
    # firstly, get the number of the items in the train random split
    DATAROOT = folder
    BOXES_PATH = os.path.join(DATAROOT, "boxes")
    MEASUREMENTS_PATH = os.path.join(DATAROOT, "measurements")

    # read number of files in the folders
    n_files = len([name for name in os.listdir(BOXES_PATH) if os.path.isfile(os.path.join(BOXES_PATH, name))])
    print("Number of files in boxes folder: ", n_files)

    # get the random split
    train_files = random.sample(range(n_files), int(n_files*0.75))
    val_files = [i for i in range(n_files) if i not in train_files]

    # order files
    train_files = sorted(train_files)
    val_files = sorted(val_files)

    # from dataroot, get only the folder name
    folder_name = os.path.basename(DATAROOT)
    print(folder_name)

    # create a train.txt and a val.txt with the files INCLUDING THE PATH TO THE FILE (WITHOUT THE EXTENSION)
    # overwrite the file if it exists
    # first delete the file if single route

    if single_route or "mini" in MASTER_DATAROOT:
        try:
            os.remove(os.path.join(MASTER_DATAROOT, "train.txt"))
        except:
            pass

        try:
            os.remove(os.path.join(MASTER_DATAROOT, "val.txt"))
        except:
            pass


    with open(os.path.join(MASTER_DATAROOT, "train.txt"), "a") as f:
        for item in train_files:
            f.write(os.path.join(DATAROOT, str(item).zfill(4)) + "\n")

    with open(os.path.join(MASTER_DATAROOT, "val.txt"), "a") as f:
        for item in val_files:
            f.write(os.path.join(DATAROOT, str(item).zfill(4)) + "\n")

            

Number of files in boxes folder:  100
route_0_routes_training_date_03_04_13_55_31


In [523]:
# # create the splits in train.txt and val.txt

# # 75% of n_files
# n_files_train = int(n_files * 0.75)
# n_files_val = n_files - n_files_train

# # train will be from 0 to 30 and val from 30 to the end
# with open(os.path.join(DATAROOT, "train.txt"), "w") as f:
#     for i in range(n_files_train):
#         f.write(str(i).zfill(4) + "\n")

# with open(os.path.join(DATAROOT, "val.txt"), "w") as f:
#     for i in range(n_files_train, n_files):
#         f.write(str(i).zfill(4) + "\n")

# # create CAM_FRONT, CAM_FRONT_LEFT, CAM_FRONT_RIGHT, CAM_BACK, CAM_BACK_LEFT, CAM_BACK_RIGHT in train and val folders
# os.makedirs(os.path.join(DATAROOT, "train", "CAM_FRONT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "train", "CAM_FRONT_LEFT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "train", "CAM_FRONT_RIGHT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "train", "CAM_BACK"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "train", "CAM_BACK_LEFT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "train", "CAM_BACK_RIGHT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "train", "lidar"), exist_ok=True)

# # move the images to the corresponding folders
# # instead of for i in range(n_files):, read from train and val files
# with open(os.path.join(DATAROOT, "train.txt"), "r") as f:
#     for i in f:
#         i = i.strip()
#         i = int(i)
#         # check if exists
#         if not os.path.exists(os.path.join(DATAROOT, "CAM_FRONT", str(i).zfill(4) + ".jpg")):
#             continue
#         os.rename(os.path.join(DATAROOT, "CAM_FRONT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "train", "CAM_FRONT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_FRONT_LEFT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "train", "CAM_FRONT_LEFT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_FRONT_RIGHT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "train", "CAM_FRONT_RIGHT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_BACK", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "train", "CAM_BACK", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_BACK_LEFT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "train", "CAM_BACK_LEFT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_BACK_RIGHT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "train", "CAM_BACK_RIGHT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "points", str(i).zfill(4) + ".bin"), os.path.join(DATAROOT, "train", "lidar", str(i).zfill(4) + ".bin"))

# # same for val
# os.makedirs(os.path.join(DATAROOT, "val", "CAM_FRONT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "val", "CAM_FRONT_LEFT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "val", "CAM_FRONT_RIGHT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "val", "CAM_BACK"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "val", "CAM_BACK_LEFT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "val", "CAM_BACK_RIGHT"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "val", "lidar"), exist_ok=True)

# # instead of for i in range(n_files):, read from train and val files
# with open(os.path.join(DATAROOT, "val.txt"), "r") as f:
#     for i in f:
#         i = i.strip()
#         i = int(i)
#         if not os.path.exists(os.path.join(DATAROOT, "CAM_FRONT", str(i).zfill(4) + ".jpg")):
#             continue
#         os.rename(os.path.join(DATAROOT, "CAM_FRONT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "val", "CAM_FRONT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_FRONT_LEFT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "val", "CAM_FRONT_LEFT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_FRONT_RIGHT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "val", "CAM_FRONT_RIGHT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_BACK", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "val", "CAM_BACK", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_BACK_LEFT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "val", "CAM_BACK_LEFT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "CAM_BACK_RIGHT", str(i).zfill(4) + ".jpg"), os.path.join(DATAROOT, "val", "CAM_BACK_RIGHT", str(i).zfill(4) + ".jpg"))
#         os.rename(os.path.join(DATAROOT, "points", str(i).zfill(4) + ".bin"), os.path.join(DATAROOT, "val", "lidar", str(i).zfill(4) + ".bin"))

# # now move calib files
# os.makedirs(os.path.join(DATAROOT, "train", "calib"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "val", "calib"), exist_ok=True)

# # instead of for i in range(n_files):, read from train and val files
# with open(os.path.join(DATAROOT, "train.txt"), "r") as f:
#     for i in f:
#         i = i.strip()
#         i = int(i)
#         if not os.path.exists(os.path.join(DATAROOT, "calib", str(i).zfill(4) + ".txt")):
#             continue
#         os.rename(os.path.join(DATAROOT, "calib", str(i).zfill(4) + ".txt"), os.path.join(DATAROOT, "train", "calib", str(i).zfill(4) + ".txt"))

# # instead of for i in range(n_files):, read from train and val files
# with open(os.path.join(DATAROOT, "val.txt"), "r") as f:
#     for i in f:
#         i = i.strip()
#         i = int(i)
#         if not os.path.exists(os.path.join(DATAROOT, "calib", str(i).zfill(4) + ".txt")):
#             continue
#         os.rename(os.path.join(DATAROOT, "calib", str(i).zfill(4) + ".txt"), os.path.join(DATAROOT, "val", "calib", str(i).zfill(4) + ".txt"))

# # now move labels
# os.makedirs(os.path.join(DATAROOT, "train", "labels"), exist_ok=True)
# os.makedirs(os.path.join(DATAROOT, "val", "labels"), exist_ok=True)

# # instead of for i in range(n_files):, read from train and val files
# with open(os.path.join(DATAROOT, "train.txt"), "r") as f:
#     for i in f:
#         i = i.strip()
#         i = int(i)
#         if not os.path.exists(os.path.join(DATAROOT, "labels", str(i).zfill(4) + ".txt")):
#             continue
#         os.rename(os.path.join(DATAROOT, "labels", str(i).zfill(4) + ".txt"), os.path.join(DATAROOT, "train", "labels", str(i).zfill(4) + ".txt"))

# # instead of for i in range(n_files):, read from train and val files
# with open(os.path.join(DATAROOT, "val.txt"), "r") as f:
#     for i in f:
#         i = i.strip()
#         i = int(i)
#         if not os.path.exists(os.path.join(DATAROOT, "labels", str(i).zfill(4) + ".txt")):
#             continue
#         os.rename(os.path.join(DATAROOT, "labels", str(i).zfill(4) + ".txt"), os.path.join(DATAROOT, "val", "labels", str(i).zfill(4) + ".txt"))